# Custom Model

In [21]:
import tensorflow as tf

import keras

from keras import layers
from keras import models
from keras import optimizers

from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import sklearn as sk

FILE_VERSION_SUFFIX = "v3"




## Dataset Import

In [22]:
cifar10 = tf.keras.datasets.cifar10
(X_train, Y_train), (X_test,Y_test) = cifar10.load_data()


X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)



In [23]:
cmi = keras.Input(shape=(None,None,3))

end_models = []

filter_counts =  [32, 16, 16, 32, 32, 64, 64]
kernel_sizes  =  [3,  13, 11,  9,  7,  5,  3]

In [ ]:


from keras.callbacks import LearningRateScheduler

dense_size = 32

for j in range(5):
    for i in range(len(filter_counts)):
        #convert X_sample to float32

        #if the model file already exists, skip it
        inner_model = None

        try:
            inner_model = models.load_model(f'model_{filter_counts[i]}_{kernel_sizes[i]}_{FILE_VERSION_SUFFIX}.keras', compile=False)
            print(f'Loaded model_{filter_counts[i]}_{kernel_sizes[i]}_{FILE_VERSION_SUFFIX}.keras')
            inner_model.trainable = True
        except:

            inner_model = models.Sequential()
            inner_model.add(layers.InputLayer(shape=(32,32,3)))

            inner_model.add(layers.Conv2D(filter_counts[i], (kernel_sizes[i], kernel_sizes[i]), activation='relu', padding='same'))
            inner_model.add(layers.Dropout(0.25))
            inner_model.add(layers.BatchNormalization())

            inner_model.add(layers.Conv2D(2*filter_counts[i], (3, 3), activation='relu', padding='same'))
            inner_model.add(layers.Dropout(0.25))
            inner_model.add(layers.BatchNormalization())
            inner_model.add(layers.MaxPooling2D((2, 2)))

            inner_model.add(layers.Conv2D(4*filter_counts[i], (3, 3), activation='relu', padding='same'))
            inner_model.add(layers.Dropout(0.25))
            inner_model.add(layers.BatchNormalization())
            inner_model.add(layers.MaxPooling2D((2, 2)))

        inner_model.add(layers.GlobalAveragePooling2D())
        inner_model.add(layers.Dense(dense_size, activation='relu'))
        inner_model.add(layers.Dense(10, activation='softmax'))

        print(f"Training model_{filter_counts[i]}_{kernel_sizes[i]}_{FILE_VERSION_SUFFIX}.keras")

        inner_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        inner_model.fit(X_train, Y_train, epochs=5, validation_data=(X_test,Y_test), verbose=1)

        inner_model.pop(rebuild=True)
        inner_model.pop(rebuild=True)
        inner_model.pop(rebuild=True)

        inner_model.save(f'model_{filter_counts[i]}_{kernel_sizes[i]}_{FILE_VERSION_SUFFIX}.keras')


Training model_32_3_v3.keras
Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 72s 45ms/step - accuracy: 0.3655 - loss: 1.7379 - val_accuracy: 0.3350 - val_loss: 2.0489
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.5112 - loss: 1.3527 - val_accuracy: 0.4008 - val_loss: 1.8053
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.5372 - loss: 1.2953 - val_accuracy: 0.4177 - val_loss: 1.6205
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.5417 - loss: 1.2996 - val_accuracy: 0.3364 - val_loss: 2.0042
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.5214 - loss: 1.3712 - val_accuracy: 0.4391 - val_loss: 1.6342
Training model_16_13_v3.keras
Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 72s 44ms/step - accuracy: 0.3101 - loss: 1.8650 - val_accuracy: 0.1856 - val_loss: 2.6412
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 69s 44ms/step - accuracy: 0.4144 - loss: 1.5856 - val_accuracy: 0.2279 - val_loss: 2.3607
Epoch 3/5
1563/1563 ━━━━

In [25]:
#load the cifar100 dataset
cifar100 = tf.keras.datasets.cifar100

(X_train, Y_train), (X_test,Y_test) = cifar100.load_data()

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

print(Y_test.shape)

the_models = []

for i in range(len(filter_counts)):
    inner_model = models.load_model(f'model_{filter_counts[i]}_{kernel_sizes[i]}_{FILE_VERSION_SUFFIX}.keras')
    print(f'Loaded model_{filter_counts[i]}_{kernel_sizes[i]}_{FILE_VERSION_SUFFIX}.keras')
    inner_model.trainable = False

    #Fine tune the model
    model = models.Sequential()
    model.add(layers.InputLayer(shape=(32,32,3)))
    model.add(inner_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(100, activation='softmax'))

    optimizer = optimizers.Adam(learning_rate=0.00005)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, Y_train, epochs=20, batch_size=64)

    model.trainable = False

    the_models.append(model)




(10000, 100)
Loaded model_32_3_v3.keras


/Users/thomasjones/.pyenv/versions/finproj/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - accuracy: 0.0286 - loss: 4.6032
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.0788 - loss: 4.1631
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.1000 - loss: 4.0165
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.1092 - loss: 3.9335
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.1226 - loss: 3.8662
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.1266 - loss: 3.8127
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.1378 - loss: 3.7716
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.1404 - loss: 3.7418
Epoch 9/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.1473 - loss: 3.7126
Epoch 10/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.1481 - loss: 3.6809
Epoch 11/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.1534 - loss: 3.6555
Epoch 12/20
782/782 ━━━━━━━━━━

In [26]:

#load the cifar100 dataset
cifar100 = tf.keras.datasets.cifar100

(X_train, Y_train), (X_test,Y_test) = cifar100.load_data()

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

#make the models trainable
for model in the_models:
    model.trainable = False


#Construct an ensemble model
inp = layers.Input(shape=(32,32,3))
x = layers.Add()([model(inp) for model in the_models])
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(100, activation='softmax')(x)

ensemble_model = models.Model(inp, x)

print(ensemble_model.summary())
print(Y_train.shape)
print(X_train.shape)

ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

ensemble_model.fit(X_train, Y_train, epochs=20, batch_size=64)

#Create a confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np

Y_pred = ensemble_model.predict(X_test)

print(Y_pred.shape)
print(Y_test.shape)

yp_max = np.argmax(Y_pred, axis=1)
yt_max = np.argmax(Y_test, axis=1)

#print the metrics
print(classification_report(yt_max, yp_max))

print(confusion_matrix(yt_max, yp_max))


Model: "functional_913"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_23      │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_10       │ (None, 100)       │    153,380 │ input_layer_23[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_11       │ (None, 100)       │     74,308 │ input_layer_23[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_12       │ (None, 100)       │     72,004 │ input_layer_23[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_13       │ (None, 100)       │    160,292 │ input_layer_23[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_14       │ (None, 100)       │    157,220 │ input_layer_23[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_15       │ (None, 100)       │    468,196 │ input_layer_23[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_16       │ (None, 100)       │    465,124 │ input_layer_23[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 100)       │          0 │ sequential_10[0]… │
│                     │                   │            │ sequential_11[0]… │
│                     │                   │            │ sequential_12[0]… │
│                     │                   │            │ sequential_13[0]… │
│                     │                   │            │ sequential_14[0]… │
│                     │                   │            │ sequential_15[0]… │
│                     │                   │            │ sequential_16[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_110 (Dense)   │ (None, 128)       │     12,928 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_111 (Dense)   │ (None, 100)       │     12,900 │ dense_110[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,576,352 (6.01 MB)

 Trainable params: 25,828 (100.89 KB)

 Non-trainable params: 1,550,524 (5.91 MB)

None
(50000, 100)
(50000, 32, 32, 3)
Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 42ms/step - accuracy: 0.1540 - loss: 3.9097
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.2472 - loss: 3.1232
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2583 - loss: 3.0338
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.2697 - loss: 2.9829
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2751 - loss: 2.9534
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.2771 - loss: 2.9264
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.2779 - loss: 2.9215
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.2836 - loss: 2.9062
Epoch 9/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2877 - loss: 2.8951
Epoch 10/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.2892 - loss: 2.8684
Epoch 11/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.2890 - loss: 

In [27]:
confusion_matrix(yt_max, yp_max)

array([[44,  0,  2, ...,  0,  0,  0],
       [ 0,  6,  2, ...,  0,  0,  2],
       [ 0,  0, 19, ...,  0,  2,  1],
       ...,
       [ 0,  0,  4, ...,  8,  0,  0],
       [ 2,  0,  6, ...,  0,  6,  2],
       [ 0,  0,  1, ...,  0,  0, 40]])